#Установка CityGeoTools

In [1]:
!git clone https://github.com/iduprojects/CityGeoTools

Cloning into 'CityGeoTools'...
remote: Enumerating objects: 2741, done.
remote: Counting objects: 100% (996/996), done.
remote: Compressing objects: 100% (378/378), done.
remote: Total 2741 (delta 665), reused 819 (delta 614), pack-reused 1745
Receiving objects: 100% (2741/2741), 113.97 MiB | 16.25 MiB/s, done.
Resolving deltas: 100% (1890/1890), done.


In [2]:
!pip install -r /content/CityGeoTools/metrics/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 97.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.0/307.0 kB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 96.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.0/243.0 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 72.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 63.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
!pip install scipy

In [4]:
!pip install rpyc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.0/74.0 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.0/127.0 kB 8.1 MB/s eta 0:00:00


In [5]:
!pip install networkit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 12.5 MB/s eta 0:00:00


In [6]:
import sys
import os
import numpy as np
import geopandas as gpd
import folium
from CityGeoTools.metrics.data import CityInformationModel as BaseModel

#Модель города

In [7]:
city_model = BaseModel.CityInformationModel(city_name="Ekaterinburg", city_crs=32640, cwd="CityGeoTools")

#Здания

In [ ]:
buildings = gpd.read_file('/content/buildings.geojson')
blocks = gpd.read_file('/content/city_blocks.geojson')

In [9]:
buildings = buildings.to_crs(epsg=32640)
blocks = blocks.to_crs(buildings.crs)

In [10]:
buildings['basement_area'] = buildings.geometry.area

In [11]:
buildings["id"] = range(len(buildings))
blocks["block_id"] = range(len(blocks))

In [12]:
buildings = buildings.sjoin(blocks, how="left")
buildings = buildings.drop(columns=["index_right"])

In [13]:
buildings["block_id"] = buildings["block_id"].fillna(buildings["block_id"].median())

In [14]:
buildings = buildings.astype({"is_living": bool, "basement_area": int, "block_id": int})

In [15]:
buildings = buildings.rename(columns={"id_left": "id", "area_residential": "living_area","building:levels": "storeys_count"})

In [16]:
buildings = buildings.to_crs(epsg=4326)
buildings.crs = 'urn:ogc:def:crs:OGC::CRS84'

columns_to_keep = ["id", "is_living", "basement_area", "living_area", "storeys_count", "block_id", "geometry"]
buildings = buildings[columns_to_keep]

buildings.to_file('Building.geojson', driver='GeoJSON')

In [17]:
city_model.update_layer(attr_name="Buildings", file_name="/content/Building.geojson")

print(city_model.Buildings)

Validation of Buildings layer...
Buildings layer loaded successfully!
                                                geometry     id  is_living  \
0      POLYGON ((721881.179 6307103.381, 721874.689 6...      0      False   
1      POLYGON ((712039.364 6286882.658, 712043.891 6...      1      False   
2      POLYGON ((712079.507 6286902.853, 712085.440 6...      2      False   
3      POLYGON ((712199.425 6286895.112, 712205.190 6...      3      False   
4      POLYGON ((721638.532 6308281.218, 721638.334 6...      4      False   
...                                                  ...    ...        ...   
47942  POLYGON ((719215.282 6286218.311, 719221.003 6...  47937      False   
47943  POLYGON ((719415.566 6285987.289, 719421.937 6...  47938      False   
47944  POLYGON ((717242.332 6298322.008, 717251.815 6...  47939      False   
47945  POLYGON ((722718.454 6301067.724, 722717.749 6...  47940      False   
47946  POLYGON ((722688.099 6301236.736, 722681.323 6...  47941      Fal

In [18]:
city_model.methods.spacematrix.message

{'Buildings': 'Layer matches specification'}

#Кварталы

In [ ]:
blocks = gpd.read_file('/content/city_blocks.geojson')
districts = gpd.read_file('/content/districts.geojson')

In [68]:
districts = districts.to_crs(epsg=32640)
blocks = blocks.to_crs(districts.crs)

In [69]:
blocks['basement_area'] = blocks.geometry.area
blocks['municipality_id'] = 10

In [70]:
blocks["block_id"] = range(len(blocks))
districts["district_id"] = range(len(districts))

In [71]:
blocks = blocks.sjoin(districts, how="left")
blocks = blocks.drop_duplicates(subset='block_id')
blocks["district_id"] = blocks["district_id"].fillna(blocks["district_id"].median())

In [72]:
blocks

,id_left,geometry,basement_area,municipality_id,block_id,index_right,id_right,name,population_2021,district_id
0,0,"POLYGON ((704898.256 6283849.149, 704862.079 6...",2.089217e+04,10,0,2.0,relation/5817295,Чкаловский район,286277.0,2.0
1,0,"POLYGON ((700089.633 6286974.858, 700089.554 6...",5.352391e+03,10,1,2.0,relation/5817295,Чкаловский район,286277.0,2.0
2,7,"POLYGON ((694547.807 6296904.118, 694547.947 6...",8.438398e+07,10,2,2.0,relation/5817295,Чкаловский район,286277.0,2.0
3,11,"POLYGON ((694905.127 6292395.107, 694905.162 6...",2.152238e+04,10,3,0.0,relation/5803327,Верх-Исетский район,240822.0,0.0
4,12,"POLYGON ((694753.884 6292885.895, 694754.018 6...",1.385853e+04,10,4,0.0,relation/5803327,Верх-Исетский район,240822.0,0.0
...,...,...,...,...,...,...,...,...,...,...
2007,2454,"POLYGON ((737949.801 6300647.553, 737949.741 6...",1.013252e+04,10,2007,1.0,relation/5803648,Октябрьский район,151775.0,1.0
2008,2455,"POLYGON ((737969.351 6300675.464, 737969.272 6...",1.983217e+04,10,2008,1.0,relation/5803648,Октябрьский район,151775.0,1.0
2009,2456,"POLYGON ((737865.583 6300876.338, 737865.528 6...",2.660361e+04,10,2009,1.0,relation/5803648,Октябрьский район,151775.0,1.0
2010,2457,"POLYGON ((736962.499 6300810.434, 736986.921 6...",3.536666e+05,10,2010,1.0,relation/5803648,Октябрьский район,151775.0,1.0


In [73]:
blocks = blocks.drop(columns=["id_left","index_right","population_2021","population_2021", "id_right", "name"])

In [74]:
blocks

,geometry,basement_area,municipality_id,block_id,district_id
0,"POLYGON ((704898.256 6283849.149, 704862.079 6...",2.089217e+04,10,0,2.0
1,"POLYGON ((700089.633 6286974.858, 700089.554 6...",5.352391e+03,10,1,2.0
2,"POLYGON ((694547.807 6296904.118, 694547.947 6...",8.438398e+07,10,2,2.0
3,"POLYGON ((694905.127 6292395.107, 694905.162 6...",2.152238e+04,10,3,0.0
4,"POLYGON ((694753.884 6292885.895, 694754.018 6...",1.385853e+04,10,4,0.0
...,...,...,...,...,...
2007,"POLYGON ((737949.801 6300647.553, 737949.741 6...",1.013252e+04,10,2007,1.0
2008,"POLYGON ((737969.351 6300675.464, 737969.272 6...",1.983217e+04,10,2008,1.0
2009,"POLYGON ((737865.583 6300876.338, 737865.528 6...",2.660361e+04,10,2009,1.0
2010,"POLYGON ((736962.499 6300810.434, 736986.921 6...",3.536666e+05,10,2010,1.0


In [75]:
blocks = blocks.rename(columns={"block_id":"id", "district_id":"administrative_unit_id", "basement_area":"area"})

In [76]:
blocks['administrative_unit_id'] = blocks['administrative_unit_id'].astype(int)

In [77]:
blocks

,geometry,area,municipality_id,id,administrative_unit_id
0,"POLYGON ((704898.256 6283849.149, 704862.079 6...",2.089217e+04,10,0,2
1,"POLYGON ((700089.633 6286974.858, 700089.554 6...",5.352391e+03,10,1,2
2,"POLYGON ((694547.807 6296904.118, 694547.947 6...",8.438398e+07,10,2,2
3,"POLYGON ((694905.127 6292395.107, 694905.162 6...",2.152238e+04,10,3,0
4,"POLYGON ((694753.884 6292885.895, 694754.018 6...",1.385853e+04,10,4,0
...,...,...,...,...,...
2007,"POLYGON ((737949.801 6300647.553, 737949.741 6...",1.013252e+04,10,2007,1
2008,"POLYGON ((737969.351 6300675.464, 737969.272 6...",1.983217e+04,10,2008,1
2009,"POLYGON ((737865.583 6300876.338, 737865.528 6...",2.660361e+04,10,2009,1
2010,"POLYGON ((736962.499 6300810.434, 736986.921 6...",3.536666e+05,10,2010,1


In [78]:
blocks = blocks.to_crs(epsg=4326)
blocks.crs = 'urn:ogc:def:crs:OGC::CRS84'

blocks.to_file("blocks.geojson", driver="GeoJSON")

In [79]:
city_model.update_layer(attr_name="Blocks", file_name="/content/blocks.geojson")

print(city_model.Blocks)

Validation of Blocks layer...
Blocks layer loaded successfully!
                                               geometry          area  \
0     POLYGON ((704898.256 6283849.149, 704862.079 6...  2.089217e+04   
1     POLYGON ((700089.633 6286974.858, 700089.554 6...  5.352391e+03   
2     POLYGON ((694547.807 6296904.118, 694547.947 6...  8.438398e+07   
3     POLYGON ((694905.127 6292395.107, 694905.162 6...  2.152238e+04   
4     POLYGON ((694753.884 6292885.895, 694754.018 6...  1.385853e+04   
...                                                 ...           ...   
2007  POLYGON ((737949.801 6300647.553, 737949.741 6...  1.013252e+04   
2008  POLYGON ((737969.351 6300675.464, 737969.272 6...  1.983217e+04   
2009  POLYGON ((737865.583 6300876.338, 737865.528 6...  2.660361e+04   
2010  POLYGON ((736962.499 6300810.434, 736986.921 6...  3.536666e+05   
2011  POLYGON ((731403.105 6302341.959, 731403.099 6...  5.244520e+07   

      municipality_id    id  administrative_unit_id  
0    

In [80]:
city_model.methods.spacematrix.message

{'Buildings': 'Layer matches specification',
 'Blocks': 'Layer matches specification'}